In [47]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
import math
from qiskit.circuit.library import MCXGate

x_reg = QuantumRegister(4,'x')
plus_eins=QuantumRegister(1,"plus_eins")
ancilla = QuantumRegister(3, "ancilla")
c = ClassicalRegister(4, 'c')
qft_invers =QFT(3, inverse = True,do_swaps= False)
qft = QFT(3, do_swaps= False)

qc=QuantumCircuit(x_reg,plus_eins,ancilla,c)

In [48]:
def increment():    
    qc.append(qft, ancilla)
    for i in range(3):
        qc.cp((2*math.pi)/2**(i+1),plus_eins,ancilla[i])
    qc.append(qft_invers, ancilla)
    
def decrement():    
    qc.append(qft, ancilla)
    for i in range(3):
        qc.cp(-(2*math.pi)/2**(i+1),plus_eins,ancilla[i])

In [49]:
qc.x(plus_eins)
qc.barrier()
qc.h(x_reg)
qc.h(plus_eins)
qc.barrier()

for _ in range(1):
    # First Clause
    qc.cx(x_reg[0],plus_eins[0])
    
    increment()
    qc.x(plus_eins[0])
    qc.barrier()
    
    # Second Clause
    qc.cx(x_reg[0],plus_eins[0])
    qc.cx(x_reg[2],plus_eins[0])
    qc.mcx(control_qubits=[x_reg[0], x_reg[2]], target_qubit=plus_eins[0])
    
    increment()
    qc.x(plus_eins[0])
    qc.barrier()
    
    # Third Clause
    qc.x(x_reg[1])
    qc.cx(x_reg[1], plus_eins[0])
    qc.x(x_reg[1])
    
    increment()
    qc.x(plus_eins[0])
    qc.barrier()
    
    # Fourth Clause
    qc.x(x_reg[3])
    qc.cx(x_reg[3], plus_eins[0])
    qc.x(x_reg[3])
    
    increment()
    qc.x(plus_eins[0])
    qc.barrier()
    
    # Fifth Clause
    qc.cx(x_reg[2], plus_eins[0])
    qc.cx(x_reg[3], plus_eins[0])
    qc.mcx(control_qubits=[x_reg[2], x_reg[3]], target_qubit=plus_eins[0])
    
    increment()
    qc.x(plus_eins[0])
    qc.barrier()
    
    #Check for 5 clauses
    qc.mcx(control_qubits=[ancilla[0], ancilla[1], ancilla[2]], target_qubit=plus_eins[0])
    qc.barrier()
    
    #Mirroing
    #Fifth Clause
    qc.cx(x_reg[2], plus_eins[0])
    qc.cx(x_reg[3], plus_eins[0])
    qc.mcx(control_qubits=[x_reg[2], x_reg[3]], target_qubit=plus_eins[0])
    
    decrement()
    qc.x(plus_eins[0])
    qc.barrier()
    
    #Fourth Clause
    qc.x(x_reg[3])
    qc.cx(x_reg[3], plus_eins[0])
    qc.x(x_reg[3])
    
    decrement()
    qc.x(plus_eins[0])
    qc.barrier()
    
    #Third Clause
    qc.x(x_reg[1])
    qc.cx(x_reg[1], plus_eins[0])
    qc.x(x_reg[1])
    
    decrement()
    qc.x(plus_eins[0])
    qc.barrier()
    
    #Second Clause
    qc.cx(x_reg[0],plus_eins[0])
    qc.cx(x_reg[2],plus_eins[0])
    qc.mcx(control_qubits=[x_reg[0], x_reg[2]], target_qubit=plus_eins[0])
    
    decrement()
    qc.x(plus_eins[0])
    qc.barrier()
    
    #First Clause
    qc.cx(x_reg[0],plus_eins[0])
    
    decrement()
    qc.x(plus_eins[0])
    qc.barrier()
    

KeyError: 'HGate'

In [40]:
qc.draw()

░ ┌───┐ ░                                            ░ »
      x_0: ──────░─┤ H ├─░────■───────────────────────────────────────░─»
                 ░ ├───┤ ░    │                                       ░ »
      x_1: ──────░─┤ H ├─░────┼───────────────────────────────────────░─»
                 ░ ├───┤ ░    │                                       ░ »
      x_2: ──────░─┤ H ├─░────┼───────────────────────────────────────░─»
                 ░ ├───┤ ░    │                                       ░ »
      x_3: ──────░─┤ H ├─░────┼───────────────────────────────────────░─»
           ┌───┐ ░ ├───┤ ░  ┌─┴─┐                             ┌───┐   ░ »
plus_eins: ┤ X ├─░─┤ H ├─░──┤ X ├───■──────■────────■─────────┤ X ├───░─»
           └───┘ ░ └───┘ ░ ┌┴───┴─┐ │P(π)  │        │       ┌─┴───┴─┐ ░ »
ancilla_0: ──────░───────░─┤0     ├─■──────┼────────┼───────┤0      ├─░─»
                 ░       ░ │      │        │P(π/2)  │       │       │ ░ »
ancilla_1: ──────░───────░─┤1 QFT ├────────■────────┼───────┤1 IQFT ├─░─»
                 ░       ░ │      │                 │P(π/4) │       │ ░ »
ancilla_2: ──────░───────░─┤2     ├─────────────────■───────┤2      ├─░─»
                 ░       ░ └──────┘                         └───────┘ ░ »
      c: 4/═════════════════════════════════════════════════════════════»
                                                                        »
«                                                                ░         »
«      x_0: ───■───────────■─────────────────────────────────────░─────────»
«              │           │                                     ░  ┌───┐  »
«      x_1: ───┼───────────┼─────────────────────────────────────░──┤ X ├──»
«              │           │                                     ░  └───┘  »
«      x_2: ───┼──────■────■─────────────────────────────────────░─────────»
«              │      │    │                                     ░         »
«      x_3: ───┼──────┼────┼─────────────────────────────────────░─────────»
«            ┌─┴─┐  ┌─┴─┐┌─┴─┐                           ┌───┐   ░         »
«plus_eins: ─┤ X ├──┤ X ├┤ X ├─■──────■────────■─────────┤ X ├───░─────────»
«           ┌┴───┴─┐└───┘└───┘ │P(π)  │        │       ┌─┴───┴─┐ ░ ┌──────┐»
«ancilla_0: ┤0     ├───────────■──────┼────────┼───────┤0      ├─░─┤0     ├»
«           │      │                  │P(π/2)  │       │       │ ░ │      │»
«ancilla_1: ┤1 QFT ├──────────────────■────────┼───────┤1 IQFT ├─░─┤1 QFT ├»
«           │      │                           │P(π/4) │       │ ░ │      │»
«ancilla_2: ┤2     ├───────────────────────────■───────┤2      ├─░─┤2     ├»
«           └──────┘                                   └───────┘ ░ └──────┘»
«      c: 4/═══════════════════════════════════════════════════════════════»
«                                                                          »
«                                                   ░ 
«      x_0: ────────────────────────────────────────░─
«                 ┌───┐                             ░ 
«      x_1: ──■───┤ X ├─────────────────────────────░─
«             │   └───┘                             ░ 
«      x_2: ──┼─────────────────────────────────────░─
«             │                                     ░ 
«      x_3: ──┼─────────────────────────────────────░─
«           ┌─┴─┐                           ┌───┐   ░ 
«plus_eins: ┤ X ├─■──────■────────■─────────┤ X ├───░─
«           └───┘ │P(π)  │        │       ┌─┴───┴─┐ ░ 
«ancilla_0: ──────■──────┼────────┼───────┤0      ├─░─
«                        │P(π/2)  │       │       │ ░ 
«ancilla_1: ─────────────■────────┼───────┤1 IQFT ├─░─
«                                 │P(π/4) │       │ ░ 
«ancilla_2: ──────────────────────■───────┤2      ├─░─
«                                         └───────┘ ░ 
«      c: 4/══════════════════════════════════════════
«